In [1]:
#!/user/bin/env python

import sys
import string
import csv
import math
import re
import xlwt
import datetime
import numpy

# from datetime import datetime

In [2]:
'''
All functions below to extract task metrics  (getExecutorRunTime, getExecutorCPURunTime, getExecutorDeserializeCPUTime,
getExecutorDeserializeTime, getShuffleWriteTime) output results in seconds for a particular task.
'''

'\nAll functions below to extract task metrics  (getExecutorRunTime, getExecutorCPURunTime, getExecutorDeserializeCPUTime,\ngetExecutorDeserializeTime, getShuffleWriteTime) output results in seconds for a particular task.\n'

In [77]:
file = "./sparkFiles/spark-events/app-20200214152759-0009"
def getStages(filename):
    stage_count = 0
    with open(filename) as openLog:
        for line in openLog:
            if '"Event":"SparkListenerStageCompleted"' in line:
                if '"Stage ID":' in line:
                    #print(line)
                    stage_count = stage_count+1
    
    #print(stage_count)

    return stage_count

getStages(file)

2

In [78]:
def getTasks(filename, stage):
    total = 0
    stage_check = '"Stage ID":'+str(stage) + ','
#     task_success_check = '{"Reason":"Success"}'
    with open(filename) as openLog:
        for line in openLog:
                if '"Event":"SparkListenerStageSubmitted"' and stage_check in line:
                        if '"Number of Tasks":' in line:
                            partB = line.split('"Number of Tasks":')[1]
                            partB = partB.split(",")
                            tasks = partB[0] 
                            total = int(tasks)

        return total
print(getTasks(file,0))

8


In [79]:
def getTaskDuration(filename, stage, tid):

    executorRunTime = getExecutorRunTime(filename, stage, tid)
    
    task_duration = executorRunTime
    task_duration = round(task_duration, 4)
    
    return task_duration

print(getTaskDuration(file, 1, 9))

2.6263


In [80]:
#to calculate total time taken by a stage
file = "./sparkFiles/spark-events/app-20200214152759-0009"
def getStageDuration(filename, stage):
    stage_duration = 0
    stage_check = '"Stage ID":' + str(stage) + ','
    start_time = ''
    finish_time = ''
    with open(filename) as openLog:
            for line in openLog:
                if '"Event":"SparkListenerStageCompleted"' in line:
                    if stage_check in line:
                        start_time = line.split('"Submission Time":')[1]
                        start_time = start_time.split(',')[0]
                        finish_time = line.split('"Completion Time":')[1]
                        finish_time = finish_time.split(',')[0]
                        break

    TimestampUtc = "\/Date(" + start_time + ")\/"
    TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    date_s = datetime.datetime.fromtimestamp(int(TimestampUtc))
#     print (date_s)
    
    TimestampUtc = "\/Date(" + finish_time + ")\/"
    TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    date_f = datetime.datetime.fromtimestamp(int(TimestampUtc))
#     print (date_f)

    duration = date_f - date_s
    duration = str(duration)
    
    duration = duration.split(':')
    
    duration_in_secs = (60*60*int(duration[0])) + (60*int(duration[1])) + int(duration[2])
    
    duration_in_mins = duration_in_secs/60
    
    return duration_in_mins

print(getStageDuration(file, 1))

4.966666666666667


In [81]:
#get parallel stages from an application
file = "./sparkFiles/spark-events/app-20190507123104-0000"

def getParallelStages(filename):
    stage_duration = 0
    start_time = ''
    finish_time = ''
    stage_id = ''
    stages = {}
    parallel_stages = {}
    parallel_stages_filetered = []
    pstages_final =[]
    
    with open(filename) as openLog:
            for line in openLog:
                if '"Event":"SparkListenerStageCompleted"' in line:
                        stage_times = []
                        start_time = line.split('"Submission Time":')[1]
                        start_time = start_time.split(',')[0]
                        
                        TimestampUtc = "\/Date(" + start_time + ")\/"
                        TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]
                        date_s = datetime.datetime.fromtimestamp(int(TimestampUtc))
                        date_s = str(date_s)
                        
                        stage_id = line.split('"Stage ID":')[1]
                        stage_id = stage_id.split(',')[0]
                        stages[stage_id] = date_s
                 
    
    
    for stage,time in stages.items():
        parallel_stages.setdefault(time, set()).add(int(stage))
    
#     print(parallel_stages)
    
    for key, values in parallel_stages.items():
        parallel_stages_filtered = [values for key, values in parallel_stages.items() 
                              if len(values) > 1]
        
    for i in parallel_stages_filtered:
        list_i = list(i)
        pstages_final += list_i

    pstages_final.sort()
    return pstages_final, parallel_stages_filtered
#     print(pstages_final)
    

print(getParallelStages(file))

([2, 3, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 32, 33, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51], [{2, 3}, {8, 7}, {10, 11}, {12, 13}, {16, 15}, {17, 18}, {20, 21}, {24, 22, 23}, {25, 26}, {27, 28}, {32, 33}, {35, 36}, {37, 38}, {40, 41}, {43, 44}, {45, 46}, {48, 47}, {50, 51}])


In [82]:
#get number of waves in a stage
def getWaves(numOfTasks, cores):
    waves = numOfTasks/cores
    waves = math.ceil(waves)
    
    return waves

In [83]:
#extract variable stages
def getScalingStages(ref1_stagetimes, ref2_stagetimes, ref1_partitions, ref2_partitions, stages):
    scalingstages = []
    for i in range(stages):
        if ref1_partitions[i] != ref2_partitions[i]:
                    scalingstages.append(i)
                
    return scalingstages 

In [84]:
#to get overall application time
def getApplicationTime(filename):
    stages = getStages(filename)
    total_application_time = 0
    
    for i in range(stages):
        stageduration = getStageDuration(filename, i)
        total_application_time = total_application_time + stageduration
        
    return total_application_time

print(getApplicationTime(file))

69.85


In [85]:
#get average times for each stage
def getAvgWaveTimes(file, cores):
    stages = getStages(file)
    avgwavetimes = []
    stagedurations = []
    wavesperstage = []

    for stage in range(stages):

        tasks = getTasks(file, stage)

        print("\nStage ", stage)
        waves = getWaves(tasks, cores)
        wavesperstage.append(waves)
        print("Number of Tasks: ", tasks)
        print("Number of Waves: ", waves)

        stagetime = getStageDuration(file, stage)

        avg_task_time = stagetime/waves

        avgwavetimes.append(avg_task_time)
        print("Stage Duration: ", stagetime)
        print("Average Wave Time: ", avg_task_time)

        stagedurations.append(stagetime)

    print("\nAverage Wave Times Per Stage")
    print(avgwavetimes)

    print("\nStage Times")
    print(stagedurations)
    
file = "./sparkFiles/spark-events/app-20190703111240-0003"
getAvgWaveTimes(file, 5)


Stage  0
Number of Tasks:  64
Number of Waves:  13
Stage Duration:  15.266666666666667
Average Wave Time:  1.1743589743589744

Stage  1
Number of Tasks:  1
Number of Waves:  1
Stage Duration:  0.0
Average Wave Time:  0.0

Stage  2
Number of Tasks:  1
Number of Waves:  1
Stage Duration:  0.016666666666666666
Average Wave Time:  0.016666666666666666

Stage  3
Number of Tasks:  64
Number of Waves:  13
Stage Duration:  10.183333333333334
Average Wave Time:  0.7833333333333333

Stage  4
Number of Tasks:  64
Number of Waves:  13
Stage Duration:  0.016666666666666666
Average Wave Time:  0.001282051282051282

Stage  5
Number of Tasks:  64
Number of Waves:  13
Stage Duration:  9.333333333333334
Average Wave Time:  0.717948717948718

Stage  6
Number of Tasks:  64
Number of Waves:  13
Stage Duration:  6.316666666666666
Average Wave Time:  0.4858974358974359

Stage  7
Number of Tasks:  1
Number of Waves:  1
Stage Duration:  0.016666666666666666
Average Wave Time:  0.016666666666666666

Stage  8
N

In [86]:
# gather reference data
ref1 = "./sparkFiles/spark-events/app-20190913185420-0030"
ref2 = "./sparkFiles/spark-events/app-20190913203944-0054"

ref_cores = 5

ref1_stagedurations = [] 
ref2_stagedurations = []

ref1_tasks = []
ref2_tasks= []

ref1_stages = getStages(ref1)
ref2_stages = getStages(ref2)

totalScalingTime1 = 0
totalScalingTime2 = 0

avgWaveTimes = []

ref1_parallelstages, ref1_parallelstagesets = getParallelStages(ref1)
ref2_parallelstages, ref2_parallelstagesets = getParallelStages(ref2)

const_parallelstagepartitions = []
parallel_stage_set = []

for i in range(ref1_stages):
    ref1_stagedurations.append(getStageDuration(ref1, i))
    ref1_tasks.append(getTasks(ref1, i))
    
for i in range(ref2_stages):
    ref2_stagedurations.append(getStageDuration(ref2, i))
    ref2_tasks.append(getTasks(ref2, i))

if not all(ref1_parallelstages):
    ParallelStages = False
else:
    ParallelStages = True

scalingStages = getScalingStages(ref1_stagedurations, ref2_stagedurations, ref1_tasks, ref2_tasks, ref1_stages)    

ParallelStages = True

if ParallelStages == False:
    print("Scaling Stages: ", scalingStages)
    partitionCounts = []
    for i in range(len(scalingStages)):
        stagetime1 = getStageDuration(ref1, scalingStages[i])
        partitions1 = getTasks(ref1, scalingStages[i])
        waves1 = getWaves(partitions1, ref_cores)
        avgTaskTime1 = stagetime1/waves1

        stagetime2 = getStageDuration(ref2, scalingStages[i])
        partitions2 = getTasks(ref2, scalingStages[i])
        waves2 = getWaves(partitions2, ref_cores)
        avgTaskTime2 = stagetime2/waves2
        average_partition_count = (partitions1 + (partitions2/2))/2
        partitionCounts.append(average_partition_count)

        avgTaskTime = (avgTaskTime1 + avgTaskTime2)/2
        avgWaveTimes.append(avgTaskTime)

        totalScalingTime1 = totalScalingTime1 + stagetime1
        totalScalingTime2 = totalScalingTime2 + stagetime2


    constTime1 = getApplicationTime(ref1) - totalScalingTime1
    constTime2 = getApplicationTime(ref2) - totalScalingTime2
    constantTime = (constTime1 + constTime2)/2
    print("Average Partition Count:", partitionCounts)
    print("Average Wave Times:", avgWaveTimes)
    print("Constant Time:", constantTime)
    print("Parallel Stage Set:", parallel_stage_set)

else:
    pstage_durations1 = []
    pstage_partitions1 = []
    pstage_durations2 = []
    pstage_partitions2 = []
    partitionCounts = []
    scaling_parallelstages1 = list(set(scalingStages).intersection(ref1_parallelstages))
    scaling_parallelstages2 = list(set(scalingStages).intersection(ref2_parallelstages))

    
    for j in range(len(scaling_parallelstages1)):
        for k in ref1_parallelstagesets:
            if scaling_parallelstages1[j] in k:
                for x in k:
                    pstage_durations1.append(getStageDuration(ref1, x))
                    pstage_partitions1.append(getTasks(ref1, x))
                break
        break
    print("Parallel Stage Group in Ref 1:", pstage_partitions1)
        
    max_pstage_duration1 = max(pstage_durations1)
    tot_pstage_partitions1 = sum(pstage_partitions1)
    waves1 = getWaves(tot_pstage_partitions1, ref_cores)
    avgTaskTime1 = max_pstage_duration1/waves1
    print("Average Task Time for Ref 1:", avgTaskTime1)
    print("Maximum parallel Stage Time for Ref 1:", max_pstage_duration1)
    print("Waves for Stage 1:", waves1)
    
    for j in range(len(scaling_parallelstages2)):
        for k in ref2_parallelstagesets:
            if scaling_parallelstages2[j] in k:
                for x in k:
                    pstage_durations2.append(getStageDuration(ref2, x))
                    pstage_partitions2.append(getTasks(ref2, x))
                break
        break
                
    print("Parallel Stage Group in Ref 2:", pstage_partitions2)
    max_pstage_duration2 = max(pstage_durations2)
    tot_pstage_partitions2 = sum(pstage_partitions2)
    
    waves2 = getWaves(tot_pstage_partitions2, ref_cores)
    avgTaskTime2 = max_pstage_duration2/waves2
    print("Average Task Time for Ref 2:", avgTaskTime2)
    print("Maximum parallel Stage Time for Ref 2:", max_pstage_duration2)
    print("Waves for Stage 2:", waves2)
    avgTaskTime = (avgTaskTime1 + avgTaskTime2)/2
    avgWaveTimes.append(avgTaskTime)
    print(pstage_durations1)
    print(sum(pstage_durations1))
    constTime1 = getApplicationTime(ref1) - sum(pstage_durations1)
    constTime2 = getApplicationTime(ref2) - sum(pstage_durations2)
    print("Constant Time 1:", constTime1)
    print("Constant Time 2:", constTime2)
    constantTime = (constTime1 + constTime2)/2
    
    avg_partition_count = math.ceil((tot_pstage_partitions1 + tot_pstage_partitions2/2)/2)
    partitionCounts.append(avg_partition_count)

    print("Average Partition Count:", avg_partition_count)
    print("Average Wave Times:", avgWaveTimes)
    print("Constant Time:", constantTime)

Parallel Stage Group in Ref 1: [2, 12, 2]
Average Task Time for Ref 1: 0.07083333333333333
Maximum parallel Stage Time for Ref 1: 0.2833333333333333
Waves for Stage 1: 4
Parallel Stage Group in Ref 2: [2, 24, 2]
Average Task Time for Ref 2: 0.09166666666666667
Maximum parallel Stage Time for Ref 2: 0.55
Waves for Stage 2: 6
[0.03333333333333333, 0.2833333333333333, 0.016666666666666666]
0.3333333333333333
Constant Time 1: 0.06666666666666665
Constant Time 2: 0.08333333333333326
Average Partition Count: 15
Average Wave Times: [0.08125]
Constant Time: 0.07499999999999996


In [87]:
#get predictions
def predictApplicationTime(_cores, _data):
    cores = _cores

    new_stage_times = []

    input_size = _data

    for s in range(len(partitionCounts)):
        newWaveNumber = getWaves(partitionCounts[s]*input_size, cores)

        new_stage_times.append(newWaveNumber*avgWaveTimes[s])

    scaledTime = sum(new_stage_times)
    predictedTime = scaledTime + constantTime

    print("Estimated Application Duration:", predictedTime)
    
predictApplicationTime(40,50)

Estimated Application Duration: 1.61875
